# Toronto Clustering Lab
## Submission for question 3 of 3

Start from previous submission state.

In [1]:
%run ./"IBM Capstone Project - Toronto Clustering Lab 2 of 3.ipynb"

Requirement already up-to-date: lxml in /opt/home_local/lib/python3.8/site-packages (4.6.2)
Requirement already up-to-date: html5lib in /opt/home_local/lib/python3.8/site-packages (1.1)
Requirement already up-to-date: beautifulsoup4 in /opt/home_local/lib/python3.8/site-packages (4.9.3)
Web scraping with pandas installed and imported successfully.
Scraped 180 rows successfully.
103
Geospatial information downloaded successfully.


## 1. Download and Explore Dataset

1.1. I will only explore the neighbourhoods in Toronto.

In [2]:
toronto_data = canada_postal_codes[canada_postal_codes.Borough.str.contains('Toronto')]

1.2. Prepare all the dependencies for this exercise.

In [3]:
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim

import requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported successfully.')

Libraries imported successfully.


1.3. Count totals from the **Toronto** dataset.

In [4]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(toronto_data['Borough'].unique()),
        toronto_data.shape[0]
    )
)

Toronto has 4 boroughs and 39 neighborhoods.


1.4. Get coordinates of Toronto.

In [5]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


1.5. Visualize Toronto with the neighbourhoods.

In [6]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

1.6. I am going to start utilizing the Foursquare API to explore the neighborhoods and segment them. Here, I will define the credentials.

In [7]:
CLIENT_ID = '5PB2NYAAGTUF0MMQSQHLEIEC3C25VCDUNAAJUQWNR4KLHJBZ' # your Foursquare ID
CLIENT_SECRET = 'T33ULAX352E5T00E4CDKYHQHX012NFZDTJMYAELGS0OKKIOF' # your Foursquare Secret
ACCESS_TOKEN = 'BMO1Q3QFN43BIWJIYVB4EH5C5HJEPZSBZVWYY2ZPHODOPP5Z' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5PB2NYAAGTUF0MMQSQHLEIEC3C25VCDUNAAJUQWNR4KLHJBZ
CLIENT_SECRET:T33ULAX352E5T00E4CDKYHQHX012NFZDTJMYAELGS0OKKIOF


1.7. Let's explore the first neighborhood in my dataframe.

In [8]:
neighborhood_name = toronto_data['Neighbourhood'].values[0]
print(neighborhood_name)

Regent Park, Harbourfront


1.8. Get the neighborhood's latitude and longitude values.

In [9]:
neighborhood_latitude = toronto_data['Latitude'].values[0]
neighborhood_longitude = toronto_data['Longitude'].values[0]

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


1.9. Create the GET request URL to obtain the top 100 venues that are in the neighbourhood within a radius of 500 meters.

In [10]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=5PB2NYAAGTUF0MMQSQHLEIEC3C25VCDUNAAJUQWNR4KLHJBZ&client_secret=T33ULAX352E5T00E4CDKYHQHX012NFZDTJMYAELGS0OKKIOF&v=20180604&ll=43.6542599,-79.3606359&radius=500&limit=30'

1.10. Send the GET request and examine the results.

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5feb6b96afd2f83d5f06eec7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 48,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

1.11. Prepare a function that will help me to get categories.

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

1.12. Clean the json and structure it into a pandas dataframe.

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues

30 venues were returned by Foursquare.


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Body Blitz Spa East,Spa,43.654735,-79.359874
5,Impact Kitchen,Restaurant,43.656369,-79.356980
6,The Extension Room,Gym / Fitness Center,43.653313,-79.359725
7,The Distillery Historic District,Historic Site,43.650244,-79.359323
8,Corktown Common,Park,43.655618,-79.356211
9,SOMA chocolatemaker,Chocolate Shop,43.650622,-79.358127


## 2. Explore Neighborhoods in Toronto